# CBF Training Analysis

Analyze training curves and model performance.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import sys
sys.path.append('..')

from src.cbf.lipschitz_network import LipschitzCBFNetwork

## Load Training History

In [ ]:
history = np.load('../models/checkpoints/training_history.npz')

train_loss = history['train_loss']
val_loss = history['val_loss']
val_accuracy = history['val_accuracy']

epochs = np.arange(1, len(train_loss) + 1)

print(f"Training epochs: {len(epochs)}")
print(f"Final validation accuracy: {val_accuracy[-1]:.4f}")
print(f"Best validation accuracy: {np.max(val_accuracy):.4f} at epoch {np.argmax(val_accuracy)+1}")

## Training Curves

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss curves
axes[0].plot(epochs, train_loss, label='Train Loss', linewidth=2)
axes[0].plot(epochs, val_loss, label='Val Loss', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training and Validation Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy curve
axes[1].plot(epochs, val_accuracy, label='Val Accuracy', linewidth=2, color='green')
axes[1].axhline(y=0.95, color='r', linestyle='--', label='95% threshold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Validation Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/figures/training_curves.png', dpi=300, bbox_inches='tight')
plt.show()

## Load Trained Model

In [ ]:
model = LipschitzCBFNetwork()
checkpoint = torch.load('../models/checkpoints/cbf_lipschitz_epoch500.pth', map_location='cpu')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print(f"Loaded model from epoch {checkpoint['epoch']}")
print(f"Lipschitz constant: {model.lipschitz_constant():.4f}")
print(f"Validation metrics: {checkpoint['metrics']}")

## Barrier Function Visualization

In [ ]:
# Create 2D slice through state space
n_points = 100
x_range = np.linspace(-1.5, 1.5, n_points)
y_range = np.linspace(-1.5, 1.5, n_points)
X, Y = np.meshgrid(x_range, y_range)

# Evaluate barrier function
H = np.zeros_like(X)

for i in range(n_points):
    for j in range(n_points):
        # Create state with varying human x,y positions
        state = np.zeros(38)
        state[12] = X[i, j]  # Human x position
        state[13] = Y[i, j]  # Human y position
        
        with torch.no_grad():
            h_val = model(torch.FloatTensor(state).unsqueeze(0)).item()
        
        H[i, j] = h_val

# Plot
plt.figure(figsize=(10, 8))
contour = plt.contourf(X, Y, H, levels=20, cmap='RdYlGn')
plt.contour(X, Y, H, levels=[0], colors='black', linewidths=3, label='Safety boundary')
plt.colorbar(contour, label='h(x)')
plt.xlabel('Human X Position (m)')
plt.ylabel('Human Y Position (m)')
plt.title('Learned Barrier Function Landscape')
plt.grid(True, alpha=0.3)
plt.savefig('../results/figures/barrier_function_landscape.png', dpi=300, bbox_inches='tight')
plt.show()